In [75]:
import pandas as pd
import numpy as np

In [76]:
cd C:\Users\home\Desktop\class\IDS 690-02\mid-term project\estimating-impact-of-opioid-prescription-regulations-team-1\20_intermediate_files

C:\Users\home\Desktop\class\IDS 690-02\mid-term project\estimating-impact-of-opioid-prescription-regulations-team-1\20_intermediate_files


In [77]:
prescriptions = pd.read_csv("arcos_grouped_all_FIPS.csv")

In [78]:
cd C:\Users\home\Desktop\class\IDS 690-02\mid-term project\estimating-impact-of-opioid-prescription-regulations-team-1\20_intermediate_files\Merged_Files

C:\Users\home\Desktop\class\IDS 690-02\mid-term project\estimating-impact-of-opioid-prescription-regulations-team-1\20_intermediate_files\Merged_Files


In [79]:
merged0 = pd.read_csv('PopMort.csv')

In [80]:
prescriptions = prescriptions.drop(columns = ['Unnamed: 0','Unnamed: 0.1']) 

mergedU=pd.merge(merged0,prescriptions,on='FIPS',how='left',validate="m:m", indicator=True)

mergedU._merge.value_counts()

both          55419
right_only        0
left_only         0
Name: _merge, dtype: int64

In [81]:
mergedU.drop(mergedU[mergedU['_merge']=='left_only'].index,inplace=True)

In [82]:
mergedU = mergedU.drop(columns = ['_merge']) 
mergedW=(mergedU[mergedU['STNAME']=='Florida'])

In [83]:
mergedW=(mergedU[mergedU['STNAME']=='Florida'])

In [84]:
mergedU['DeathsPC']=merged0['Deaths']/(merged0['POPULATION']/10000)

In [85]:
mergedW['Policy']=0
mergedW.Policy[mergedW['Year']>=2010]=1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [86]:
mergedU['Policy']=0
mergedU.Policy[mergedU['Year']>=2010]=1
mergedU['State'] = [1 if x == 'Florida' else 0 for x in mergedU['STNAME']]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [87]:
mergedU.head()

,Year,Deaths,FIPS,STNAME,CTYNAME,POPULATION,STATE,COUNTY,YEAR,MME_Str,DeathsPC,Policy,State
0,2003,11.0,22001,Louisiana,Acadia Parish,59194.0,LA,ACADIA,2006,22401.553115,1.858296,0,0
1,2003,11.0,22001,Louisiana,Acadia Parish,59194.0,LA,ACADIA,2007,25689.157670,3.863988,0,0
2,2003,11.0,22001,Louisiana,Acadia Parish,59194.0,LA,ACADIA,2008,24689.478841,3.139354,0,0
3,2003,11.0,22001,Louisiana,Acadia Parish,59194.0,LA,ACADIA,2009,28592.010975,3.126954,0,0
4,2003,11.0,22001,Louisiana,Acadia Parish,59194.0,LA,ACADIA,2010,33011.664509,1.790044,0,0


In [88]:
mergedU['Policy:State'] = mergedU['Policy'] * mergedU['State']

In [89]:
from plotnine import *
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [90]:
did_Florida = smf.ols('DeathsPC ~  Policy + Policy:State',data = mergedU).fit()
print(did_Florida.summary())

                            OLS Regression Results                            
Dep. Variable:               DeathsPC   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     3.331
Date:                Sun, 10 Nov 2019   Prob (F-statistic):             0.0358
Time:                        15:14:44   Log-Likelihood:                -11849.
No. Observations:                7917   AIC:                         2.370e+04
Df Residuals:                    7914   BIC:                         2.373e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        1.5599      0.018     88.058   

In [91]:
mergedU['adj_year'] = mergedU['Year'] - 2010
mergedU['Policy:adj_year'] = mergedU['Policy'] * mergedU['adj_year']
mergedU['State:adj_year'] = mergedU['State'] * mergedU['adj_year']
mergedU['Policy:State:adj_year'] = mergedU['Policy'] * mergedU['State'] * mergedU['adj_year']

In [92]:
did2_Florida = smf.ols('DeathsPC ~  adj_year + Policy + Policy:State + Policy:adj_year + State:adj_year + Policy:State:adj_year',data = mergedU).fit()
print(did2_Florida.summary())

                            OLS Regression Results                            
Dep. Variable:               DeathsPC   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     5.061
Date:                Sun, 10 Nov 2019   Prob (F-statistic):           3.42e-05
Time:                        15:14:45   Log-Likelihood:                -11837.
No. Observations:                7917   AIC:                         2.369e+04
Df Residuals:                    7910   BIC:                         2.374e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 1.63

In [93]:
mergedU['Policy_T']=0
mergedU.Policy_T[mergedU['Year']>=2007]=1
mergedU['State_T'] = [1 if x == 'Texas' else 0 for x in mergedU['STNAME']]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [94]:
mergedU['Policy_T:State_T'] = mergedU['Policy_T'] * mergedU['State_T']
did_Texas = smf.ols('DeathsPC ~  Policy_T + Policy_T:State_T',data = mergedU).fit()
print(did_Texas.summary())

                            OLS Regression Results                            
Dep. Variable:               DeathsPC   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     14.29
Date:                Sun, 10 Nov 2019   Prob (F-statistic):           6.40e-07
Time:                        15:19:18   Log-Likelihood:                -11838.
No. Observations:                7917   AIC:                         2.368e+04
Df Residuals:                    7914   BIC:                         2.370e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            1.5137      0.025  

In [95]:
mergedU['adj_year_T'] = mergedU['Year'] - 2007
mergedU['Policy_T:adj_year_T'] = mergedU['Policy_T'] * mergedU['adj_year_T']
mergedU['State_T:adj_year_T'] = mergedU['State_T'] * mergedU['adj_year_T']
mergedU['Policy_T:State_T:adj_year_T'] = mergedU['Policy_T'] * mergedU['State_T'] * mergedU['adj_year_T']

In [96]:
did2_Texas = smf.ols('DeathsPC ~  adj_year_T + Policy_T + Policy_T:State_T + Policy_T:adj_year_T + State_T:adj_year_T + Policy_T:State_T:adj_year_T',data = mergedU).fit()
print(did2_Texas.summary())

                            OLS Regression Results                            
Dep. Variable:               DeathsPC   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     10.02
Date:                Sun, 10 Nov 2019   Prob (F-statistic):           4.70e-11
Time:                        15:21:31   Log-Likelihood:                -11823.
No. Observations:                7917   AIC:                         2.366e+04
Df Residuals:                    7910   BIC:                         2.371e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

In [97]:
mergedU['Policy_W']=0
mergedU.Policy_W[mergedU['Year']>=2012]=1
mergedU['State_W'] = [1 if x == 'Washington' else 0 for x in mergedU['STNAME']]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [98]:
mergedU['Policy_W:State_W'] = mergedU['Policy_W'] * mergedU['State_W']
did_Washington = smf.ols('DeathsPC ~  Policy_W + Policy_W:State_W',data = mergedU).fit()
print(did_Washington.summary())

                            OLS Regression Results                            
Dep. Variable:               DeathsPC   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     7.009
Date:                Sun, 10 Nov 2019   Prob (F-statistic):           0.000910
Time:                        15:23:21   Log-Likelihood:                -11846.
No. Observations:                7917   AIC:                         2.370e+04
Df Residuals:                    7914   BIC:                         2.372e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            1.5592      0.015  

In [99]:
mergedU['adj_year_W'] = mergedU['Year'] - 2012
mergedU['Policy_W:adj_year_W'] = mergedU['Policy_W'] * mergedU['adj_year_W']
mergedU['State_W:adj_year_W'] = mergedU['State_W'] * mergedU['adj_year_W']
mergedU['Policy_W:State_W:adj_year_W'] = mergedU['Policy_W'] * mergedU['State_W'] * mergedU['adj_year_W']

In [100]:
did2_Washington = smf.ols('DeathsPC ~  adj_year_W + Policy_W + Policy_W:State_W + Policy_W:adj_year_W + State_W:adj_year_W + Policy_W:State_W:adj_year_W',data = mergedU).fit()
print(did2_Washington.summary())

                            OLS Regression Results                            
Dep. Variable:               DeathsPC   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     4.241
Date:                Sun, 10 Nov 2019   Prob (F-statistic):           0.000287
Time:                        15:25:12   Log-Likelihood:                -11840.
No. Observations:                7917   AIC:                         2.369e+04
Df Residuals:                    7910   BIC:                         2.374e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

In [101]:
mergedU['PrescribePC']=mergedU['MME_Str']/(mergedU['POPULATION'])

In [103]:
did_Florida_Pre= smf.ols('PrescribePC ~  Policy + Policy:State',data = mergedU).fit()
print(did_Florida_Pre.summary())

                            OLS Regression Results                            
Dep. Variable:            PrescribePC   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     543.2
Date:                Sun, 10 Nov 2019   Prob (F-statistic):          2.34e-234
Time:                        15:30:47   Log-Likelihood:                -5490.0
No. Observations:               55419   AIC:                         1.099e+04
Df Residuals:                   55416   BIC:                         1.101e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.4417      0.002    263.774   

In [104]:
did2_Florida_Pre = smf.ols('PrescribePC ~  adj_year + Policy + Policy:State + Policy:adj_year + State:adj_year + Policy:State:adj_year',data = mergedU).fit()
print(did2_Florida_Pre.summary())

                            OLS Regression Results                            
Dep. Variable:            PrescribePC   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     417.3
Date:                Sun, 10 Nov 2019   Prob (F-statistic):               0.00
Time:                        15:31:34   Log-Likelihood:                -4803.3
No. Observations:               55419   AIC:                             9621.
Df Residuals:                   55412   BIC:                             9683.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 0.42